In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyswarms
from pyswarms.single.global_best import GlobalBestPSO
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

data = pd.read_csv('/Users/veda/Downloads/Bank_Personal_Loan_Modelling.csv')

In [4]:
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
data.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [6]:
x = data['Personal Loan']
data = data.drop(['Personal Loan'],axis=1)
data = data.join(x)

In [7]:
data.isnull().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
Personal Loan         0
dtype: int64

In [8]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y[:, np.newaxis]).toarray()

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

((4000, 11), (1000, 11), (4000, 2), (1000, 2))

In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
def PSO_model(nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=13, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
n_layers = 1
model = PSO_model(4, n_layers)
model.summary()

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weight in weights_layer:
        shapes.append(weight.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [ ]:
def evaluate(W, shape,X_train=X_train, Y_train=Y_train):
    results = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_train, Y_train, verbose=0)
        results.append(1-score[1])
    return results

shape = get_shape(model)
x_max = 1.0 * np.ones(66)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.6, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=50, dimensions=66,
                          options=options, bounds=bounds)
cost, pos = optimizer.optimize(evaluate_nn, 20, X_train=X_train, Y_train=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])